# The SQuaSH InfluxDB data model

SQuaSH uses [Chronograf](https://docs.influxdata.com/chronograf/v1.8/) to visualize metrics collected by the [`lsst.verify` framework](https://sqr-019.lsst.io/). In this notebook we show how we designed the SQuaSH data model in InfluxDB and demonstrate the ETL process to get SQuaSH metrics into InfluxDB.

In practice, every time a verification job is sent to SQuaSH a background task is responsible for transforming it to the to InfluxDB line protocol and loading it to InfluxDB.

This notebook can be used to recreate the SQuaSH database in InfluxDB if needed, for example, if there's a change in the SQuaSH InfluxDB data model.


Here some important InfluxDB concepts to keep in mind:

  * InfluxDB is optimized for time-series data. In InfluxDB the data is indexed and sharded by the timestamp.
  * An InfluxDB measurement is equivalent to a table in a relational database;
  * An InfluxDB tag is equivalent to an indexed column. Tags are typically metadata used in query predicates.
  * An InfluxDB field corresponds to a non-indexed column, fields typically have the main quantity you are interested in, like the metrics in this context.
  
  
In the following cells,  we'll show how to grab the verification jobs from the SQuaSH API, transform and write them to InfluxDB using the the [line protocol](https://docs.influxdata.com/influxdb/v1.8/write_protocols/line_protocol_tutorial/):


```#<measurement>[,<tag_key>=<tag_value>[,<tag_key>=<tag_value>]] <field_key>=<field_value>[,<field_key>=<field_value>] [<timestamp>]```

## Mapping lsst.verify concepts to InfluxDB

The data model created in InfluxDB is based on the following mapping of `lsst.verify` concepts to InfluxDB concepts:

  * lsst.verify package -> InfluxDB measurement;
  * lsst.verify metadata -> InfluxDB tags or fields (see next section);
  * lsst.verify metric name -> InfluxDB field key;
  * lsst.verify metric value -> InfluxDB field value;
  * CI or LDF pipeline runtime -> InfluxDB timestamp.
  
**Note**: we avoid using the "lsst.verify measurement" terminology and use "lsst.verify metric value" instead to avoid collision with "InfluxDB measurement" which has another meaning.

### Mapping SQuaSH metadata to InfluxDB
Using `lsst.verify` the pipeline developer can add metadata to the verification jobs uploaded to SQuaSH. These metadata add information about the context where a given metric is computed, like information about the execution environment, the dataset being processed, etc. Typically job metadata is mapped to InfluxDB tags. However, there are situations where we want to map job metadata to InfluxDB fields. 

The mapping of job metadata to either InfluxDB tags or fields is defined below.

1. By default, if a SQuaSH metadata key is not found in the mapping, the original key name is preserved and it is mapped to an InfluxDB tag.
2. If the SQuaSH metadata key is found in the mapping, the `schema` key specifies the mapping. Accepted values are `tag`, `field` or `None`. If `schema` is set to `None`, then the metadata is dropped and won't be written to InfluxDB.
3. You can use the mapping to rename metadata keys when appropriate. 
4. Finally, it is also possible to define an optional transformation on the metadata value (see below).

In [ ]:
# %load ../src/squash/tasks/utils/mapping.yaml
# Defines the mapping from SQuaSH to InfluxDB.
#
# Each element is a SQuaSH key mapped to an InfluxDB key, schema specfies if
# the key is written as an InfluxDB tag or field. It is possible to specify
# a transformation for the value too (optional).
#
# By default, is the SQuaSH key is not present in the mapping
# it is added to InfluxDB as a tag using the same name as the SQuaSH key.
# If the InlfuxDB key is left blank, it means the that the SQuaSH key should
# not be used.

ci_id:
  key: run_id
  schema: field
  transformation:

run_id:
  key: run_id
  schema: field
  transformation:

id:
  key: squash_id
  schema: field
  transformation:

url:
  key: squash_url
  schema: field
  transformation: "Formatter.format_link(data['id'], value)"

# Uniformize the name for execution environment URL
run_id_url:
  key: run_url
  schema: field
  transformation: "Formatter.format_link(data['run_id'], value)"

ci_url:
  key: run_url
  schema: field
  transformation: "Formatter.format_link(data['ci_id'], value)"

# Uniformize the name for the processed dataset
ci_dataset:
  key: dataset
  schema: tag
  transformation:

version_tag:
  key: version_tag
  schema: field
  transformation:

physical_filter:
  key: filter
  schema: tag
  transformation:

filter_name:
  key: filter
  schema: tag
  transformation:

date_created:
  key: timestamp
  schema: field
  transformation: "Formatter.format_timestamp(value)"

date:
  key:
  schema:
  transformation:

ci_label:
  key:
  schema:
  transformation:

ci_name:
  key: pipeline
  schema: tag
  transformation:

code_changes:
  key: code_changes
  schema: field
  transformation: "self.format_code_changes(data['ci_id'], data['ci_name'])"

code_changes_counts:
  key: code_changes_counts
  schema: field
  transformation: "self.format_code_changes_counts(data['ci_id'], data['ci_name'])"

packages:
  key:
  schema:
  transformation:


The rationale for this mapping is the following:
1. The reason for mapping  IDs and corresponding URLs to InfluxDB fields is to [reduce InfluxDB series cardinality](https://docs.influxdata.com/influxdb/v1.7/concepts/schema_and_data_layout/#discouraged-schema-design). We also want to track these information (for example show the IDs in a Chronograf table) so they must be stored as fields.
2. We define here a unified data model for mapping metadada from different execution environments. For example, `ci_id` and `run_id` are mapped to `run_id`. `ci_dataset` and `dataset` to `dataset`. These tags don't need to have different names, since the corresponding values can be filtered by the `env_name` tag. 
3. In the current version of InfluxDB it is not possible to do [math operations with timestamps](https://community.influxdata.com/t/math-operations-on-field-value-and-time/6323/4) so it is useful to add the `timestamp` explicitly as a field. 
4. `lsst.verify` metadata uses `filter_name`. We decided to rename it to `filter` which is the name of the data ID used in DM pipeline software.
5. `ci_label` is not important so we skip that.
6. `ci_name` is mapped to `pipeline`. It identifies the pipeline that was executed. Some pipelines can have multiple verification packages.
7. We added to the SQuaSH datamodel in InfluxDB the code changes data for pipelines that run in the Jenkins environment.
8. We ignore the `packages` metadata for now. 
9. Note that, by default, keys not listed in this mapping are automatically mapped to InfluxDB tags.


See also [InfluxDB schema design and data layout](https://docs.influxdata.com/influxdb/v1.8/concepts/schema_and_data_layout/#general-recommendations) for general recommendations on designing the InfluxDB schema.



## Sending verification jobs from the SQuaSH API to InfluxDB

Query the SQuaSH API to read the verifications jobs, transform and write them to InfluxDB.

In [ ]:
SQUASH_API_URL = "https://squash-sandbox.lsst.codes/"

In [ ]:
import requests
from squash.tasks.utils.transformation import Transformer

response = requests.get(SQUASH_API_URL + "/jobs").json()
ids = response['ids']
number_of_jobs = len(ids) 
print(f"Found {number_of_jobs} jobs in the SQuaSH API")

These environment variables define the InfluxDB instance.

In [ ]:
INFLUXDB_API_URL = "https://squash-sandbox.lsst.codes/influxdb"
INFLUXDB_DATABASE = "squash-sandbox"
INFLUXDB_USERNAME = "admin"
INFLUXDB_PASSWORD = ""

Create the destination InfluxDB database if it does not exist. 

In [ ]:
from squash.tasks.influxdb import create_influxdb_database

status_code = create_influxdb_database(INFLUXDB_DATABASE, INFLUXDB_API_URL, INFLUXDB_USERNAME, INFLUXDB_PASSWORD)
status_code

Transform jobs to InfluxDB line protocol format.

In [ ]:
influxdb_lines = []
for id in ids:
    
    data = requests.get(SQUASH_API_URL + "/job/{}".format(id)).json()

    dataset = data['ci_dataset']
    
    print(f'Transforming job {id}, dataset {dataset} to InfluxDB line protocol format.')

    transformer = Transformer(squash_api_url=SQUASH_API_URL, data=data)

    influxdb_lines.extend(transformer.to_influxdb_line())

Write to InfluxDB

In [ ]:
from squash.tasks.influxdb import write_influxdb_line

for line in influxdb_lines:
    status_code = write_influxdb_line(line, INFLUXDB_DATABASE, INFLUXDB_API_URL, INFLUXDB_USERNAME, INFLUXDB_PASSWORD)
    if status_code != 204:
        print(line)